# Загрузка Pandas и очистка данных

In [2420]:
from numpy import inf
from sklearn import metrics
# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import re
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer, LabelEncoder
from collections import Counter
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2421]:
df = pd.read_csv('main_task.csv')
df.columns = df.columns.str.lower()

In [2422]:
# Почистим столбец кухонь

def clean_name(str_val):
    """
    Преобразует строку с названиями кухонь в список [list] названий кухонь.
    На входе:
        - строковая переменная, содержащая названия кухонь.
    На выходе:
        - список [list] названий кухонь.
    """
    if pd.isna(str_val):
        return str_val
    str_val = str_val.strip('[]')
    str_val = str_val.replace("\'", '')
    return str_val


df["cuisine style_1"] = df["cuisine style"].apply(clean_name)

# Заполним пропуски самыми популярными кухнями

max_count_cusine = Counter(df["cuisine style_1"].str.cat(sep=',').split(','))
new_dc = dict(sorted(max_count_cusine.items(),
                     key=lambda x: x[1], reverse=True))
new_dc
pop_cuis = ', '.join(list(new_dc.keys())[:3])

df["cuisine style_1"] = df["cuisine style_1"].fillna(pop_cuis)

# Посчитам количество кухонь в каждом ресторане
df['n cousins'] = df["cuisine style_1"].apply(lambda x: len(x.split(',')))

# Удалим дублирующий столбец
df = df.drop('cuisine style', axis=1)

In [2423]:
# Заполним пропуски в столбце Number of reviews. Найдем среднее число

mean_review = round(df['number of reviews'].mean(), 0)
df['number of reviews'] = df['number of reviews'].fillna(mean_review)

In [2424]:
# Поиск даты отзыва и создание колонки разности времени между отзывами
df = df.copy()
pattern = re.compile('\d+\/\d+\/\d+')
df['reviews_date'] = df['reviews'].apply(pattern.findall)
df['reviews_date'] = df['reviews_date'].apply(
    lambda x: [pd.to_datetime(i).date() for i in x])

df['first_review'] = df['reviews_date'].apply(
    lambda x: x[0] if len(x) == 2 else 0)
df['second_review'] = df['reviews_date'].apply(
    lambda x: x[1] if len(x) == 2 else 0)

df['delta_time'] = abs(df['second_review']-df['first_review'])
df['delta,days'] = df['delta_time'].apply(
    lambda x: 0 if type(x) == int else x.days)

# Найдем выбросы в данном признаке и заменим их на среднее значеие

#plot = df['delta,days'].hist(bins=100)


def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))[0]


outliers = outliers_iqr(list(df['delta,days']))

df['delta,days'] = round(df['delta,days'].replace(
    outliers, df['delta,days'].mean()))
#plot_1 = df['delta,days'].hist(bins=100)

df = df.drop(['delta_time', 'first_review', 'second_review',
              'reviews_date', 'reviews'], axis=1)

In [2425]:
# Присвоим цене числовое значение

price = []

# Сначала заполним пропуски самым часто встречающимся
df['price range'] = df['price range'].fillna(df['price range'].mode()[0])

for i in range(0, len(df['price range'])):
    if df['price range'].iloc[i] == '$':
        price.append(1)
    elif df['price range'].iloc[i] == '$$ - $$$':
        price.append(2)
    else:
        price.append(3)

df['price_r'] = price

In [2426]:
# создадим новый признак - относительная посещаемость - отношение количества отзывов к количеству отзывов в городе

# посчитаем сумму отзывов в каждом городе
summ_review = dict(df.groupby(['city'])['number of reviews'].sum())

# функция отношения количества отзывов к суммарному количеству отзывов


def rest_review(row, summ):
    for i in summ.keys():
        if row['city'] == i:
            res = row['number of reviews']/summ[i]*1000
    return res


df['rest_review'] = df.apply(lambda x: rest_review(x, summ_review), axis=1)

In [2427]:
# тоже самое сделаем для ранга  - отношение ранга ресторана к количеству ресторанов по городу

rest = dict(df['city'].value_counts())


def rest_rank(row, summ):
    for i in summ.keys():
        if row['city'] == i:
            res = row['ranking']/rest[i]
    return res


df['rest_rank'] = df.apply(lambda x: rest_rank(x, rest), axis=1)

In [2428]:
# Создадим новый признак - зависимость ранга ресторана от количества кухонь (логично предположить,
# что людям интереснее ходить в ресторан c большим количеством кухонь )

df['rank/cousins'] = df['ranking']/df['n cousins']

In [2429]:
# Еще одним логичным признаком является скорость набора ранга
# Так как в пизнаке разность дней сть 0, то заменим бесконечность в признаке
# скорость набора рейтинга на среднее количество дней
df['delta,days'] = df['delta,days'].replace(0, df['delta,days'].mean())
df['velocity ranking'] = df['ranking']/df['delta,days']

In [2430]:
# создадим новый пизнак, обозначающий разницу в процентах стоимости еды  ресторане от средней стоимости еды в ресторане по по городу.

mean_price = dict(df.groupby(['city'])['price_r'].mean())


def rest_rank(row, mean):
    for i in mean.keys():
        if row['city'] == i:
            res = (row['price_r'] - mean[i])
    return res


df['price_delta'] = df.apply(lambda x: rest_rank(x, mean_price), axis=1)

In [2431]:
# добавим Dummy-переменную в виде City.
df = pd.get_dummies(df, columns=['city'])

In [2432]:
df.copy()
# Удалим ткстовые данные
df['restaurant_id'] = df['restaurant_id'].str.replace('id_', '').astype(int)
df['restaurant_id']

for column in df.columns:
    if df[column].dtype == "O":
        df = df.drop(columns=column)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [2433]:
X = df.drop(['restaurant_id', 'rating'], axis=1)
y = df['rating']
RANDOM_SEED = 42

In [2434]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=RANDOM_SEED)

# Создаём, обучаем и тестируем модель

In [2435]:
regr = RandomForestRegressor(
    n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)


regr.fit(X_train, y_train)


y_pred = regr.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    6.6s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


In [2436]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21249749999999998
